# Importando as bibliotecas

In [13]:
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados
#!python -m pip install --user scikit-mlm

from skmlm import NN_MLM, MLMC

#from skmlm import FCM_MLM as NN_MLM

# Dataset

In [14]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [15]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        seed = i
        X_train, X_test, y_train, y_test = procData.holdout(0.2, seed)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação de parametros, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = NN_MLM(random_state=seed)
        grid_mlm = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_mlm.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        rp_number_best = grid_mlm.best_estimator_.rp_number

        MLM = NN_MLM(rp_number = rp_number_best, random_state=seed)
        MLM.fit(X_train, y_train)

        #testando o modelo
        y_pred = MLM.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        # f2-score
        # Fbeta = ((1 + beta^2) * Precision * Recall) / (beta^2 * Precision + Recall)
        f1_score = 5 * (precision * recall) / (4 * precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = {"rp_number": rp_number_best};
        resultados_parametros.append(best_parametros)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    resultados_gerais['params'] = resultados_parametros
    
    return resultados_gerais

In [16]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [17]:
# pontos de referencia
rp_number = np.arange(0.7,0.8,0.1)
param_grid = {'rp_number': rp_number}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [18]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('MLM',dict_metricas, rodadas, salvarResultados)

[[24  4]
 [ 3 89]]
Rodada: # 0
{'rp_number': 383}
Accuracy: 0.9416666666666667
Precision: 0.956989247311828
Recall: 0.967391304347826
F1-Score: 0.9652928416485901


[[23 10]
 [ 2 85]]
Rodada: # 1
{'rp_number': 336}
Accuracy: 0.9
Precision: 0.8947368421052632
Recall: 0.9770114942528736
F1-Score: 0.9593679458239279


[[17  5]
 [ 5 93]]
Rodada: # 2
{'rp_number': 383}
Accuracy: 0.9166666666666666
Precision: 0.9489795918367347
Recall: 0.9489795918367347
F1-Score: 0.9489795918367347


[[29  3]
 [ 3 85]]
Rodada: # 3
{'rp_number': 383}
Accuracy: 0.95
Precision: 0.9659090909090909
Recall: 0.9659090909090909
F1-Score: 0.9659090909090908


[[26  2]
 [ 4 88]]
Rodada: # 4
{'rp_number': 336}
Accuracy: 0.95
Precision: 0.9777777777777777
Recall: 0.9565217391304348
F1-Score: 0.9606986899563318


[[19 13]
 [ 3 85]]
Rodada: # 5
{'rp_number': 383}
Accuracy: 0.8666666666666667
Precision: 0.8673469387755102
Recall: 0.9659090909090909
F1-Score: 0.9444444444444444


[[15  7]
 [ 3 95]]
Rodada: # 6
{'rp_number'